In [3]:
pip install youtube_transcript_api

Note: you may need to restart the kernel to use updated packages.


In [4]:
from youtube_transcript_api import YouTubeTranscriptApi
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
link = "https://www.youtube.com/watch?v=Y8Tko2YC5hA"

In [6]:
id = link.split("=")[1]

In [7]:
id

'Y8Tko2YC5hA'

In [8]:
sub = YouTubeTranscriptApi.get_transcript(id)

In [9]:
len(sub)

62

In [10]:
sub[0]

{'text': "In this video, I'm going to answer the top 3 questions",
 'start': 0.0,
 'duration': 4.0}

In [11]:
sub[0]['text']

"In this video, I'm going to answer the top 3 questions"

In [12]:
subtitle = " ".join(x['text'] for x in sub)

In [13]:
subtitle

'In this video, I\'m going to answer the top 3 questions my students ask me about Python. What is Python? What  can you do with it? And why is it so popular? In other words, what does it do that other programming languages don\'t? Python is the  world\'s fastest growing and most popular programming language, not just  amongst software engineers, but also amongst mathematicians,  data analysts, scientists, accountants, networking engineers, and even kids! Because it\'s a very beginner friendly programming  language. So people from different disciplines use Python for a variety of different tasks, such as data analysis and visualization,  artificial intelligence and machine learning, automation  in fact this is one of the big uses of Python amongst people who are not software developers. If you constantly have to do boring, repetitive  tasks, such as copying files and folders around, renaming them,  uploading them to a server, you can easily write a Python script to automate all that and

In [14]:
import string
string.punctuation  #FOR REMOVEING PUNCTUATION WE USE STRING MODULE

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:

subtitle2 = " ";
for x in subtitle:
    if x not in string.punctuation:
       subtitle2 = "".join(x)
       
     
    

In [16]:
new_subtitle = subtitle.translate(str.maketrans('','',string.punctuation)) #punctuation removing Technique :::: 

In [17]:
len(new_subtitle)

3979

In [18]:
from nltk.tokenize import sent_tokenize

In [19]:
# new_subtitle = new_subtitle.replace('n','')

In [20]:
new_subtitle

'In this video Im going to answer the top 3 questions my students ask me about Python What is Python What  can you do with it And why is it so popular In other words what does it do that other programming languages dont Python is the  worlds fastest growing and most popular programming language not just  amongst software engineers but also amongst mathematicians  data analysts scientists accountants networking engineers and even kids Because its a very beginner friendly programming  language So people from different disciplines use Python for a variety of different tasks such as data analysis and visualization  artificial intelligence and machine learning automation  in fact this is one of the big uses of Python amongst people who are not software developers If you constantly have to do boring repetitive  tasks such as copying files and folders around renaming them  uploading them to a server you can easily write a Python script to automate all that and save your time And thats just on

In [21]:
sentences = sent_tokenize(new_subtitle)

In [22]:
type(sentences)

list

In [23]:
organized_sent = {k:v for v,k in enumerate(sentences)} #organize the sentencee to dictonary with corresponding key value

In [24]:
organized_sent

{'In this video Im going to answer the top 3 questions my students ask me about Python What is Python What  can you do with it And why is it so popular In other words what does it do that other programming languages dont Python is the  worlds fastest growing and most popular programming language not just  amongst software engineers but also amongst mathematicians  data analysts scientists accountants networking engineers and even kids Because its a very beginner friendly programming  language So people from different disciplines use Python for a variety of different tasks such as data analysis and visualization  artificial intelligence and machine learning automation  in fact this is one of the big uses of Python amongst people who are not software developers If you constantly have to do boring repetitive  tasks such as copying files and folders around renaming them  uploading them to a server you can easily write a Python script to automate all that and save your time And thats just o

In [25]:
tf_idf = TfidfVectorizer()

In [26]:
sentence_vectors = tf_idf.fit_transform(sentences)

In [27]:
sentence_vectors

<1x298 sparse matrix of type '<class 'numpy.float64'>'
	with 298 stored elements in Compressed Sparse Row format>

In [28]:
import numpy as np

In [29]:
sent_scores = np.array(sentence_vectors.sum(axis=1)).ravel()

In [30]:
sent_scores

array([9.58070645])

In [31]:
N = 3
top_n_sentences = [sentences[index] for index in np.argsort(sent_scores, axis=0)[::-1][:N]]

In [32]:
top_n_sentences

['In this video Im going to answer the top 3 questions my students ask me about Python What is Python What  can you do with it And why is it so popular In other words what does it do that other programming languages dont Python is the  worlds fastest growing and most popular programming language not just  amongst software engineers but also amongst mathematicians  data analysts scientists accountants networking engineers and even kids Because its a very beginner friendly programming  language So people from different disciplines use Python for a variety of different tasks such as data analysis and visualization  artificial intelligence and machine learning automation  in fact this is one of the big uses of Python amongst people who are not software developers If you constantly have to do boring repetitive  tasks such as copying files and folders around renaming them  uploading them to a server you can easily write a Python script to automate all that and save your time And thats just o

In [33]:
from joblib import dump, load

In [35]:
dump(summarizer,open('summarizer.joblib','wb'))

NameError: name 'summarizer' is not defined

In [44]:

def summarizer(text):
    import spacy
    from spacy.lang.en.stop_words import STOP_WORDS
    from heapq import nlargest
    from string import punctuation
    print("ll")
    stop_words = list(STOP_WORDS)
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    tokens = [token.text for token in doc]
    word_freq = {}

    for word in doc:
        if word.text.lower() not in stop_words and word.text.lower() not in punctuation:
            if word.text not in word_freq.keys():
                word_freq[word.text] = 1
            else:
                word_freq[word.text] += 1

    max_freq = max(word_freq.values())
    for word in word_freq:
        word_freq[word] = word_freq[word]/max_freq

    sent_token = [sent for sent in doc.sents]

    sent_scores = {}

    for sent in sent_token:
        for word in sent:
            if word.text in word_freq.keys():
                if sent not in sent_scores.keys():
                    sent_scores[sent] = word_freq[word.text]
                else:
                    sent_scores[sent] += word_freq[word.text]

    select_len = int(len(sent_token)*0.3)
    summary = nlargest(select_len, sent_scores, key=sent_scores.get)
    final_summary = [word.text for word in summary]
    summary = ' '.join(final_summary)

    return summary

In [48]:
dump(summarizer,open('summarizer.joblib','wb'))

In [50]:
summarizer = load(open('summarizer.joblib', 'rb'))

In [51]:
summarizer("This is a simple form. Now you will want to call a fetch on the button click or the form submit. I personally used the button click but it is a matter of preference. So I will create a function and pass it into an onClick on the button. I also use async and await, which is also a preference")

ll


'Now you will want to call a fetch on the button click or the form submit.'